In [1]:
from pyflink.table import EnvironmentSettings, TableEnvironment, DataTypes, CsvTableSource
import datetime
from pyflink.table.expressions import col
from pyflink.table.window import Over, GroupWindow
from pyflink.table.expressions import col, UNBOUNDED_RANGE, CURRENT_RANGE
from pyflink.table.udf import udf
# create a batch TableEnvironment
env_settings = EnvironmentSettings.in_batch_mode()
table_env = TableEnvironment.create(env_settings)

In [2]:
# InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
# 536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/01/2010 8:26,2.55,17850,United Kingdom
column_names = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 
                'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']

column_types = [DataTypes.STRING(), DataTypes.STRING(), DataTypes.STRING(), DataTypes.DOUBLE(), 
                DataTypes.STRING(), DataTypes.DOUBLE(),DataTypes.STRING(), DataTypes.STRING()]

source = CsvTableSource(
   '../data/1k-dataset.csv',  
    column_names,
    column_types,
    ignore_first_line=True,
    quote_character='"',
    lenient=True
)

# source is data source -origin data is from
# table is flink api
# sink is target where data goes to after transformation
table_env.register_table_source('invoices', source)

# invoices is a table
invoices = table_env.from_path('invoices')


##############################
print('\nRegistered Tables List')
print(table_env.list_tables())

print('\nFinancial Trxs Schema')
invoices.print_schema()



Registered Tables List
['invoices']

Financial Trxs Schema
(
  `InvoiceNo` STRING,
  `StockCode` STRING,
  `Description` STRING,
  `Quantity` DOUBLE,
  `InvoiceDate` STRING,
  `UnitPrice` DOUBLE,
  `CustomerID` STRING,
  `Country` STRING
)


In [3]:
invoices.fetch(3).execute().print()

+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+
|                      InvoiceNo |                      StockCode |                    Description |                       Quantity |                    InvoiceDate |                      UnitPrice |                     CustomerID |                        Country |
+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+
|                         536365 |                         85123A | WHITE HANGING HEART T-LIGHT... |                            6.0 |                12/01/2010 8:26 |                           2.55 |   

In [4]:
#         .column_by_expression("rowtime", "CAST(f0 AS TIMESTAMP(3))") \

# datetime in string format for may 25 1999
input = '12/01/2010 8:26'
  
# format
format = '%d/%m/%Y %H:%M'
  
# convert from string format to datetime format
dt = datetime.datetime.strptime(input, format)
  
# get the date from the datetime using date() 
# function
print(dt)

new_format = '%Y-%m-%d %H:%M:%S'   # SQL Format, which flink can understand
dt.strftime(new_format)

2010-01-12 08:26:00


'2010-01-12 08:26:00'

In [5]:
# UDF - User Defined Function, python/scala/java
# UDF code is not optimized by flink, python udf shall run on Python VM ie python runtime
def convertDateFormat2(input):
    # format
    format = '%d/%m/%Y %H:%M'
  
    # convert from string format to datetime format
    dt = datetime.datetime.strptime(input, format)
    new_format = '%Y-%m-%d %H:%M:%S' 
    return dt.strftime(new_format)

# creating a python function as UDF, first parameter is date string, return type is date string
convertDateFormat = udf(convertDateFormat2, [DataTypes.STRING()], DataTypes.STRING())

In [6]:
# convert data time string to SQL date time string format as new column CInvoiceDate
# drop Orignal InvoiceDate which is string 
# Cast CInvoiceDate which is SQL date time stirng to TIMESTAMP(3) 3 means precision and 
# create new Column InvoiceDate which TIMESTAMP type
# drop temp col CInvoiceDate
invoices2 = invoices.add_columns(convertDateFormat(col('InvoiceDate')).alias('CInvoiceDate'))\
                    .drop_columns(col('InvoiceDate'))\
                    .add_columns(col("CInvoiceDate").cast(DataTypes.TIMESTAMP(3)).alias("InvoiceDate"))\
                    .drop_columns(col('CInvoiceDate'))\
                    .add_columns( (col("Quantity") * col("UnitPrice")).alias("Amount"))

invoices2.print_schema()

(
  `InvoiceNo` STRING,
  `StockCode` STRING,
  `Description` STRING,
  `Quantity` DOUBLE,
  `UnitPrice` DOUBLE,
  `CustomerID` STRING,
  `Country` STRING,
  `InvoiceDate` TIMESTAMP(3),
  `Amount` DOUBLE
)


In [7]:
# DIY
# register invoices2 as table name invoices2
# in table environment, run a select query and display the output

In [8]:
table_env.register_table('invoices2', invoices2)


In [10]:
result = table_env.sql_query("SELECT InvoiceNo, Quantity, UnitPrice, Amount from invoices2")
result.fetch(20).execute().print()

+--------------------------------+--------------------------------+--------------------------------+--------------------------------+
|                      InvoiceNo |                       Quantity |                      UnitPrice |                         Amount |
+--------------------------------+--------------------------------+--------------------------------+--------------------------------+
|                         536446 |                            8.0 |                           3.75 |                           30.0 |
|                         536446 |                            6.0 |                           1.25 |                            7.5 |
|                         536446 |                           12.0 |                           2.55 |             30.599999999999998 |
|                         536446 |                           24.0 |                           1.25 |                           30.0 |
|                         536446 |                            

In [13]:
results = invoices2.group_by ( col("InvoiceNo") )\
                   .select( col("InvoiceNo"), 
                            col("Quantity").sum.alias("Qty"),
                            col("Amount").sum.alias("TotalAmount"),
                            col("InvoiceNo").count.alias("UniqueItems") )

results.execute().print()

+--------------------------------+--------------------------------+--------------------------------+----------------------+
|                      InvoiceNo |                            Qty |                    TotalAmount |          UniqueItems |
+--------------------------------+--------------------------------+--------------------------------+----------------------+
|                         536446 |                          329.0 |                         440.89 |                   32 |
|                         536466 |                           18.0 |                           42.9 |                    2 |
|                         536500 |                          102.0 |             223.90000000000003 |                   15 |
|                         536508 |                          216.0 |                         155.52 |                    2 |
|                         536514 |                          118.0 |             295.50000000000006 |                    5 |
|       

In [20]:
result = table_env.sql_query("""
SELECT InvoiceNo, sum(Quantity)  as Qty, sum(Amount) as TotalAmount, 
        count(InvoiceNo) as UniqueItems
        
FROM invoices2
GROUP BY InvoiceNo
""")

result.fetch(5).execute().print()
result.print_schema()

# source table
table_env.register_table(  "analytics_results_source", result)

+--------------------------------+--------------------------------+--------------------------------+----------------------+
|                      InvoiceNo |                            Qty |                    TotalAmount |          UniqueItems |
+--------------------------------+--------------------------------+--------------------------------+----------------------+
|                         536393 |                            8.0 |                           79.6 |                    1 |
|                         536395 |                          260.0 |              507.8800000000001 |                   14 |
|                         536401 |                          124.0 |              354.2299999999999 |                   64 |
|                         536402 |                          116.0 |                          357.0 |                    3 |
|                         536416 |                          110.0 |                          225.7 |                    6 |
+-------

In [18]:
# create a sink table that prints the output, later will cover mysql/postgres/kafka
table_env.execute_sql("""
CREATE TABLE invoice_temp_sink (
  InvoiceNo STRING,
  Qty DOUBLE,
  TotalAmount DOUBLE,
  UniqueItems  BIGINT NOT NULL
) WITH (
    'connector' = 'print'
)
""")

In [21]:
# now take results from analytics_results_source table and write to invoice_temp_sink
table_env.execute_sql("INSERT INTO invoice_temp_sink SELECT * FROM analytics_results_source").wait()

1> +I[536416, 110.0, 225.7, 6]
4> +I[536412, 220.0, 514.4099999999999, 81]
1> +I[536437, 1004.0, 842.12, 6]
4> +I[536423, 78.0, 303.90000000000003, 9]
1> +I[536367, 83.0, 278.73, 12]
1> +I[536372, 12.0, 22.200000000000003, 2]
1> +I[536374, 32.0, 350.4, 1]
1> +I[536375, 88.0, 259.86, 16]
4> +I[536425, 137.0, 362.45000000000005, 17]
1> +I[536381, 198.0, 449.97999999999996, 35]
1> +I[C536379, -1.0, -27.5, 1]
4> +I[536365, 40.0, 139.12, 7]
1> +I[536385, 53.0, 130.85, 7]
4> +I[536366, 12.0, 22.200000000000003, 2]
1> +I[536387, 1440.0, 3193.92, 5]
4> +I[536373, 88.0, 259.86, 16]
1> +I[536388, 108.0, 226.14, 14]
4> +I[536376, 112.0, 328.8, 2]
1> +I[536390, 1568.0, 1825.74, 24]
4> +I[536384, 190.0, 489.6, 13]
1> +I[C536391, -132.0, -141.48000000000002, 7]
1> +I[536393, 8.0, 79.6, 1]
1> +I[536395, 260.0, 507.8800000000001, 14]
1> +I[536401, 124.0, 354.2299999999999, 64]
1> +I[536402, 116.0, 357.0, 3]
1> +I[536463, 12.0, 17.4, 1]
1> +I[536464, 160.0, 277.34999999999997, 85]
1> +I[536488, 72.0, 1

In [22]:
# create a sink table that prints the output, later will cover mysql/postgres/kafka
table_env.execute_sql("""
CREATE TABLE invoice_temp_sink_mysql (
  InvoiceNo STRING,
  Qty DOUBLE,
  TotalAmount DOUBLE,
  UniqueItems  BIGINT NOT NULL
) WITH (
    'connector' = 'print'
)
""")

# blackhole connector doesn't print anything, does nothing with data, but ignore it
table_env.execute_sql("""
CREATE TABLE invoice_temp_sink_kafka (
  InvoiceNo STRING,
  Qty DOUBLE,
  TotalAmount DOUBLE,
  UniqueItems  BIGINT NOT NULL
) WITH (
    'connector' = 'blackhole'
)
""")

# we have 3 sinks invoice_temp_sink, invoice_temp_sink_mysql, invoice_temp_sink_kafka

In [23]:
# use statementsets to load and process analytisc once and write results into mulitple destination
# create             statementsets
statement_set = table_env.create_statement_set()

# statement set uisng python using python api

# Emit the result into sink table

statement_set.add_insert("invoice_temp_sink", result) # result is table from analytics query, print console
statement_set.add_insert("invoice_temp_sink_mysql", result) # result is table from analytics query, print console
statement_set.add_insert("invoice_temp_sink_kafka", result) # result is table from analytics query, blackhole, doesn't print

# now execute statement set, this load csv, perform analytics, insert data into all 3 sink tables
statement_set.execute().wait()

4> +I[536398, 150.0, 426.56000000000006, 17]
4> +I[536398, 150.0, 426.56000000000006, 17]
4> +I[536399, 12.0, 22.200000000000003, 2]
4> +I[536399, 12.0, 22.200000000000003, 2]
4> +I[536400, 12.0, 17.4, 1]
4> +I[536400, 12.0, 17.4, 1]
4> +I[536406, 96.0, 353.14000000000004, 17]
4> +I[536406, 96.0, 353.14000000000004, 17]
4> +I[536407, 12.0, 22.200000000000003, 2]
4> +I[536407, 12.0, 22.200000000000003, 2]
4> +I[536408, 558.0, 783.1099999999999, 48]
4> +I[536408, 558.0, 783.1099999999999, 48]
4> +I[536365, 40.0, 139.12, 7]
4> +I[536365, 40.0, 139.12, 7]
4> +I[536366, 12.0, 22.200000000000003, 2]
4> +I[536366, 12.0, 22.200000000000003, 2]
1> +I[536416, 110.0, 225.7, 6]
2> +I[536396, 102.0, 376.36000000000007, 18]
4> +I[536373, 88.0, 259.86, 16]
1> +I[536416, 110.0, 225.7, 6]
1> +I[536437, 1004.0, 842.12, 6]
1> +I[536437, 1004.0, 842.12, 6]
2> +I[536396, 102.0, 376.36000000000007, 18]
4> +I[536373, 88.0, 259.86, 16]
1> +I[536393, 8.0, 79.6, 1]
1> +I[536393, 8.0, 79.6, 1]
1> +I[536395, 260.

In [27]:
# Now statement set using Flink SQL
# we can mix flink sql and table api
# create statemetn set

# source table: analytics_results_source
# sink target tables   invoice_temp_sink, invoice_temp_sink_mysql, invoice_temp_sink_kafka
statement_set = table_env.create_statement_set()

statement_set.add_insert_sql("INSERT INTO invoice_temp_sink SELECT * FROM analytics_results_source")
statement_set.add_insert_sql("INSERT INTO invoice_temp_sink_mysql SELECT * FROM analytics_results_source")
statement_set.add_insert_sql("INSERT INTO invoice_temp_sink_kafka SELECT * FROM analytics_results_source")

print(statement_set.explain())
# execute the statement set 
statement_set.execute().wait()


== Abstract Syntax Tree ==
LogicalSink(table=[default_catalog.default_database.invoice_temp_sink], fields=[InvoiceNo, Qty, TotalAmount, UniqueItems])
+- LogicalProject(InvoiceNo=[$0], Qty=[$1], TotalAmount=[$2], UniqueItems=[$3])
   +- LogicalAggregate(group=[{0}], Qty=[SUM($1)], TotalAmount=[SUM($2)], UniqueItems=[COUNT($0)])
      +- LogicalProject(InvoiceNo=[$0], Quantity=[$3], Amount=[*($3, $5)])
         +- LogicalTableScan(table=[[default_catalog, default_database, invoices, source: [CsvTableSource(read fields: InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country)]]])

LogicalSink(table=[default_catalog.default_database.invoice_temp_sink_mysql], fields=[InvoiceNo, Qty, TotalAmount, UniqueItems])
+- LogicalProject(InvoiceNo=[$0], Qty=[$1], TotalAmount=[$2], UniqueItems=[$3])
   +- LogicalAggregate(group=[{0}], Qty=[SUM($1)], TotalAmount=[SUM($2)], UniqueItems=[COUNT($0)])
      +- LogicalProject(InvoiceNo=[$0], Quantity=[$3], Amount=[*($3, $5)])

In [26]:
print(statement_set.explain())